## Write iris.csv to a Kafka topic

In [ ]:
from kafka import KafkaProducer
import avro
from avro import schema
from avro import io
import io, random
import csv

producer = KafkaProducer(bootstrap_servers=['kafka:9092'])
 
topic = "iris"

schema = avro.schema.Parse('{"type":"record","name":"Iris","namespace":"org.apache.samoa.avro.iris","fields":[{"name":"sepallength","type":"double"},{"name":"sepalwidth","type":"double"},{"name":"petallength","type":"double"},{"name":"petalwidth","type":"double"},{"name":"class","type":{"type":"enum","name":"Labels","symbols":["Iris-setosa","Iris-versicolor","Iris-virginica"]}}]}')

with open('/home/jovyan/work/datasets/iris-dataset/iris.data', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    writer = avro.io.DatumWriter(schema)
    for row in reader:
        bytes_writer = io.BytesIO()
        encoder = avro.io.BinaryEncoder(bytes_writer)
        writer.write({"sepallength":float(row[0]), "sepalwidth":float(row[1]), "petallength":float(row[2]), "petalwidth":float(row[3]), "class":row[4]}, encoder)
        raw_bytes = bytes_writer.getvalue()
        producer.send(topic, raw_bytes)

## Read iris.csv from a Kafka topic

In [ ]:
from kafka import KafkaConsumer
import avro
from avro import schema
from avro import io
import io
import csv

topic = 'iris'

consumer = KafkaConsumer(topic,
                         bootstrap_servers=['kafka:9092'],
                         auto_offset_reset='smallest',
                         consumer_timeout_ms=2000)

schema = avro.schema.Parse('{"type":"record","name":"Iris","namespace":"org.apache.samoa.avro.iris","fields":[{"name":"sepallength","type":"double"},{"name":"sepalwidth","type":"double"},{"name":"petallength","type":"double"},{"name":"petalwidth","type":"double"},{"name":"class","type":{"type":"enum","name":"Labels","symbols":["Iris-setosa","Iris-versicolor","Iris-virginica"]}}]}')

with open('iris.csv', 'w+') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['sepallength','sepalwidth','petallength','petalwidth','class'])
    for msg in consumer:
        bytes_reader = io.BytesIO(msg.value)
        decoder = avro.io.BinaryDecoder(bytes_reader)
        reader = avro.io.DatumReader(schema)
        row = reader.read(decoder)
        writer.writerow([row['sepallength'], row['sepalwidth'], row['petallength'], row['petalwidth'], row['class']])
        